In [19]:
#進mongodb
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://10.120.30.8:27017')
db = client['music']  #選擇database
collect = db['tempo_beats']  #選擇database.collection
dic = {'musicname':'aaa','staff_id':2,'instrument':'flute','sig':'4/4','tempo':120,'start':1,'end':4,'beat':'0.25,0;0.5,1;0.125,1;0.125,1'}
collect.insert_one(dic) #放一筆dic

In [49]:
    #查詢mongoDB
client = MongoClient('mongodb://10.120.30.8:27017')
db = client['music']  
collect = db['tempo_beats']
cur = collect.find({'staff_id':1,'timesig':'4/4','start':1,'end':4},{'_id':0,'staff_id':1,'beats':1,'timesig':'4/4'}) 


In [2]:
from decimal import*
beatDic = {'measure':None,
           'whole':Decimal(1),
           'half':Decimal(1)/2,
           'quarter':Decimal(1)/4,
           'eighth':Decimal(1)/8,
           '16th':Decimal(1)/16,
           '32nd':Decimal(1)/32,
           '64th':Decimal(1)/64,
           '128th':Decimal(1)/128,
           '256th':Decimal(1)/256
          }
BDic = dict((v,k) for k,v in beatDic.iteritems())
print BDic

{Decimal('0.03125'): '32nd', Decimal('0.25'): 'quarter', Decimal('0.5'): 'half', Decimal('1'): 'whole', Decimal('0.00390625'): '256th', Decimal('0.015625'): '64th', None: 'measure', Decimal('0.0625'): '16th', Decimal('0.125'): 'eighth', Decimal('0.0078125'): '128th'}


In [4]:
# 將被合併過的拍子還原成符合四小節的拍子!，ex (3,0)=(1:0)、(1:0)、(1:0);(1.7:1) =(1:1)、(0.7:1)
def splitM(Mli,bag):
    if sum(bag) == LenMea:
        
        A = bag.pop()
        Mli.append({A:p})
        return Mli
    elif sum(bag) > LenMea:
        least = sum(bag)-LenMea
        A = bag.pop()
        Nk = A-least
        Mli.append({Nk:p})
        bag = []
        bag.append(least)
        return splitM(Mli,bag)
    elif sum(bag) < LenMea :
        Mli.append({bag[-1]:p})
        return Mli


In [5]:
from bs4 import BeautifulSoup as bs
def Rest(sigN,sigD):
#     Sig = xml.select('Measure')[0]
#     Dz = int(Sig.select('sigN')[0].text)
#     Dn = int(Sig.select('sigD')[0].text)
    dur = bs("<duration/>")
    dur.duration['n']=sigN
    dur.duration['z']=sigD
    rest = dur.duration.wrap(dur.new_tag("Rest"))
    durT = dur.new_tag("durationType")
    durT.string = "measure"
    dur.duration.insert_after(durT)
    return rest


In [6]:
def subRest(x):
    Rtag = bs("")
    Trest = Rtag.append(Rtag.new_tag("Rest")) 
    durT = Rtag.new_tag("durationType")
    durT.string = x[1]
    Rtag.Rest.append(durT)
    return Rtag

In [75]:
def Tchord(x):
    ccc = bs("")
    if (x[0][0] == 0 and x[1]=='whole'):
        Rest(sigN,sigD)
        ccc.append(Rest(sigN,sigD))
        
    elif x[0][0]==0:
        subRest(x)
        ccc.append(subRest(x))

    else:
        Ctag = bs("")
        Tcho = Ctag.append(Ctag.new_tag("Chord"))
        if x[2] >0:
            Tdot = Ctag.new_tag("dots")
            Tdot.string = str(x[2])
            Ctag.Chord.append(Tdot)
        durT = Ctag.new_tag("durationType")
        durT.string = x[1]
        Ctag.Chord.append(durT)
        for i in range(0,len(x[0])):
            Tnote = Ctag.Chord.append(Ctag.new_tag("Note"))
            Tpitch = Ctag.new_tag("pitch")
            Tpitch.string= str(x[0][i])
            Ctag.select('Note')[i].append(Tpitch)
            Ttpc = Ctag.new_tag("tpc")
            Ttpc.string="22"
            Ctag.select('Note')[i].append(Ttpc)
            Tvelo = Ctag.new_tag("velocity")
            Tvelo.string="100"
            Ctag.select('Note')[i].append(Tvelo)
            TvT = Ctag.new_tag("veloType")
            TvT.string="user"
            Ctag.select('Note')[i].append(TvT)
            ccc.append(Ctag)
    return ccc

In [63]:
#Mdic = cur[0]
#test try
Mdic = {u'staff_id': 1, u'timesig': u'4/4', u'beats': u'3,0;0.1875,72;0.0625,72;0.125,0;0.125,72;0.0625,72;0.0625,72;0.0625,72;0.0625,72;0.25,72'}

sigN , sigD = testMdic['timesig'].split('/')
sigN=int(sigN);sigD=int(sigD);Ts = sigN/sigD

LenMea =Ts*128                        #每小節的長度乘上128
Mli=[];bag=[]                         # bag=[] for def slpitM()
for beat in testMdic['beats'].split(';'):
    b = beat.split(',').pop(0) ; Bbeat = Decimal(b)*128
    p = beat.split(',')[1:]
    bag.append(Bbeat)
    splitM(Mli,bag)
# print Mli
bag=[];Measure=[[],[],[],[]];i=0
for s in Mli:
    bag.append(s.keys()[0])
    Measure[i].append(s)
    if sum(bag) == LenMea:
        i+=1
        bag=[]
# print Measure
     #change type
addB =[[],[],[],[]]
 #附點(dot)=1 可用2/3還原成音符，dot=2 可用4/7還原成音符,dot=3 可用2/3還原成音符
dotslist =[Decimal(2)/3,Decimal(4)/7,Decimal(8)/15] 
for i in range(0,4):
    AA = Measure[i]
    for Bitem in range(0,len(AA)):
        Rbeat = AA[Bitem].keys()[0]
        R = Rbeat / 128
        if R in BDic:
            dot = 0
            addB[i].append([AA[Bitem][Rbeat],BDic[R],dot])
        elif [x for x in [Decimal(2)/3,Decimal(4)/7,Decimal(8)/15] if R*x in BDic]:
            DD =  [x for x in dotslist if R*x in BDic]
            dot = dotslist.index(DD[0])+1
            addB[i].append(AA[Bitem][Rbeat],BDic[DD[0]*R],dot)
print addB 

[[[['60', '64', '67'], 'quarter', 0], [['60', '64', '67'], 'quarter', 0], [['62', '65', '70'], 'quarter', 0], [['62', '65', '70'], 'quarter', 0]], [[['60', '65', '69'], 'quarter', 0], [['60', '65', '69'], 'quarter', 0], [['62', '65', '70'], 'quarter', 0], [['62', '65', '70'], 'quarter', 0]], [], []]


In [55]:
songstr = """C,C,Bb,Bb,F,F,Bb,Bb"""
songstr = songstr.split(',')
chord = {
    'C':'60,64,67','Dm':'62,65,69','Em':'64,67,611',
    'F':'60,65,69','G':'62,67,71','Am':'60,64,69',
    'Bb':'62,65,70','Bdim':'62,65,71'
}
testMdic ={}; aa =''
sc = map(lambda a: chord[a] if a in chord else a , songstr)
for i in range(0,len(sc)):
    aa += str(Decimal(1)/4) +','+str(sc[i])+';'
ta = aa[:-1]
testMdic ={'timesig':'4/4','beats':ta}
print testMdic

{'beats': '0.25,60,64,67;0.25,60,64,67;0.25,62,65,70;0.25,62,65,70;0.25,60,65,69;0.25,60,65,69;0.25,62,65,70;0.25,62,65,70', 'timesig': '4/4'}


In [64]:
metadata ="""<?xml version="1.0" encoding="UTF-8"?>
<museScore version="2.06">
<programVersion>2.0.2</programVersion>
<programRevision>f51dc11</programRevision>
<Score>

<Part>
<Staff id="1">        
<StaffType group="pitched">          
</StaffType>        
</Staff>      
<Instrument>        
<Channel>          
<program value="5"/>          
<controller ctrl="10" value="63"/>          
<synti>Fluid</synti>          
</Channel>        
</Instrument>      
</Part>

<Part>
<Staff id="2">
<StaffType group="percussion">
<name>perc5Line</name>
<keysig>0</keysig>
</StaffType>
<defaultClef>PERC</defaultClef>
</Staff>
<Instrument>
        <Drum pitch="35">
          <head>0</head>
          <line>7</line>
          <voice>1</voice>
          <name>Acoustic Bass Drum</name>
          <stem>2</stem>
          </Drum>
        <Drum pitch="36">
          <head>0</head>
          <line>7</line>
          <voice>1</voice>
          <name>Bass Drum 1</name>
          <stem>2</stem>
          <shortcut>B</shortcut>
          </Drum>
        <Drum pitch="37">
          <head>1</head>
          <line>3</line>
          <voice>0</voice>
          <name>Side Stick</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="38">
          <head>0</head>
          <line>3</line>
          <voice>0</voice>
          <name>Acoustic Snare</name>
          <stem>1</stem>
          <shortcut>A</shortcut>
          </Drum>
        <Drum pitch="40">
          <head>0</head>
          <line>3</line>
          <voice>0</voice>
          <name>Electric Snare</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="41">
          <head>0</head>
          <line>5</line>
          <voice>0</voice>
          <name>Low Floor Tom</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="42">
          <head>1</head>
          <line>-1</line>
          <voice>0</voice>
          <name>Closed Hi-Hat</name>
          <stem>1</stem>
          <shortcut>G</shortcut>
          </Drum>
        <Drum pitch="43">
          <head>0</head>
          <line>5</line>
          <voice>1</voice>
          <name>High Floor Tom</name>
          <stem>2</stem>
          </Drum>
        <Drum pitch="44">
          <head>1</head>
          <line>9</line>
          <voice>1</voice>
          <name>Pedal Hi-Hat</name>
          <stem>2</stem>
          <shortcut>F</shortcut>
          </Drum>
        <Drum pitch="45">
          <head>0</head>
          <line>2</line>
          <voice>0</voice>
          <name>Low Tom</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="46">
          <head>1</head>
          <line>1</line>
          <voice>0</voice>
          <name>Open Hi-Hat</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="47">
          <head>0</head>
          <line>1</line>
          <voice>0</voice>
          <name>Low-Mid Tom</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="48">
          <head>0</head>
          <line>0</line>
          <voice>0</voice>
          <name>Hi-Mid Tom</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="49">
          <head>1</head>
          <line>-2</line>
          <voice>0</voice>
          <name>Crash Cymbal 1</name>
          <stem>1</stem>
          <shortcut>C</shortcut>
          </Drum>
        <Drum pitch="50">
          <head>0</head>
          <line>0</line>
          <voice>0</voice>
          <name>High Tom</name>
          <stem>1</stem>
          <shortcut>E</shortcut>
          </Drum>
        <Drum pitch="51">
          <head>1</head>
          <line>0</line>
          <voice>0</voice>
          <name>Ride Cymbal 1</name>
          <stem>1</stem>
          <shortcut>D</shortcut>
          </Drum>
        <Drum pitch="52">
          <head>1</head>
          <line>-3</line>
          <voice>0</voice>
          <name>Chinese Cymbal</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="53">
          <head>2</head>
          <line>0</line>
          <voice>0</voice>
          <name>Ride Bell</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="54">
          <head>2</head>
          <line>2</line>
          <voice>0</voice>
          <name>Tambourine</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="55">
          <head>1</head>
          <line>-3</line>
          <voice>0</voice>
          <name>Splash Cymbal</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="56">
          <head>3</head>
          <line>1</line>
          <voice>0</voice>
          <name>Cowbell</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="57">
          <head>1</head>
          <line>-3</line>
          <voice>0</voice>
          <name>Crash Cymbal 2</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="59">
          <head>1</head>
          <line>2</line>
          <voice>0</voice>
          <name>Ride Cymbal 2</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="63">
          <head>1</head>
          <line>4</line>
          <voice>0</voice>
          <name>Open Hi Conga</name>
          <stem>1</stem>
          </Drum>
        <Drum pitch="64">
          <head>1</head>
          <line>6</line>
          <voice>0</voice>
          <name>Low Conga</name>
          <stem>1</stem>
          </Drum>
<Channel>
<program value="16"/>
<controller ctrl="0" value="1"/>
<synti>Fluid</synti>
</Channel>
</Instrument>
</Part>

</Score>
</museScore>"""


In [76]:
Meta = bs(metadata, "xml")
Meta.Score.append( Meta.new_tag("Staff",id =1))

NewM = Meta.select("Score > Staff")[0]
for i in range(0,4):
    addB[i]
#     print addB[i]
    NewM.append(Meta.new_tag("Measure",number = i+1))
    measure = Meta.select("Measure")[i]
    if i == 0:
        measure.append(Meta.new_tag('KeySig'))
        adent = Meta.new_tag('accidental')
        adent.string="3"
        measure.KeySig.append(adent)
        measure.append(Meta.new_tag('TimeSig'))
        sN = Meta.new_tag('sigN')
        sN.string=str(sigN)
        measure.TimeSig.append(sN)
        sD = Meta.new_tag('sigD')
        sD.string=str(sigD)
        measure.TimeSig.append(sD)
        scs = Meta.new_tag('showCourtesySig')
        scs.string="1"
        measure.TimeSig.append(scs)
        measure.append(Meta.new_tag('Tempo'))
        temp = Meta.new_tag('tempo')
        temp.string ="4"
        measure.Tempo.append(temp)
    for a in range(0,len(addB[i])):
        measure.append(Tchord(addB[i][a]))
    print Meta.prettify()
    
with open("musicmaker.mscx", "wb") as file:
    file.write(Meta.prettify())
print Meta.prettify()

<?xml version="1.0" encoding="utf-8"?>
<museScore version="2.06">
 <programVersion>
  2.0.2
 </programVersion>
 <programRevision>
  f51dc11
 </programRevision>
 <Score>
  <Part>
   <Staff id="1">
    <StaffType group="pitched">
    </StaffType>
   </Staff>
   <Instrument>
    <Channel>
     <program value="5"/>
     <controller ctrl="10" value="63"/>
     <synti>
      Fluid
     </synti>
    </Channel>
   </Instrument>
  </Part>
  <Part>
   <Staff id="2">
    <StaffType group="percussion">
     <name>
      perc5Line
     </name>
     <keysig>
      0
     </keysig>
    </StaffType>
    <defaultClef>
     PERC
    </defaultClef>
   </Staff>
   <Instrument>
    <Drum pitch="35">
     <head>
      0
     </head>
     <line>
      7
     </line>
     <voice>
      1
     </voice>
     <name>
      Acoustic Bass Drum
     </name>
     <stem>
      2
     </stem>
    </Drum>
    <Drum pitch="36">
     <head>
      0
     </head>
     <line>
      7
     </line>
     <voice>
      1
     </

In [104]:

#Mdic = cur[0]
#test try
Mdic = {u'staff_id': 1, u'timesig': u'4/4', u'beats': u'3,0;0.1875,1;0.0625,1;0.125,0;0.125,1;0.0625,0;0.0625,1;0.0625,1;0.0625,1;0.25,1'}

sigN , sigD = Mdic['timesig'].split('/')
sigN=int(sigN);sigD=int(sigD);Ts = sigN/sigD

LenMea =Ts*128                        #每小節的長度乘上128
Mli=[];bag=[]                         # bag=[] for def slpitM()
for beat in  Mdic['beats'].split(';'):
    b , p =beat.split(',') ; Bbeat = Decimal(b)*128
    bag.append(Bbeat)
    splitM(Mli,bag)
# print Mli
bag=[];Measure=[[],[],[],[]];i=0
for s in Mli:
    bag.append(s.keys()[0])
    Measure[i].append(s)
    if sum(bag) == LenMea:
        i+=1
        bag=[]
# print Measure
     #change type
addB =[[],[],[],[]]
 #附點(dot)=1 可用2/3還原成音符，dot=2 可用4/7還原成音符,dot=3 可用8/15還原成音符
dotslist =[Decimal(2)/3,Decimal(4)/7,Decimal(8)/15]
#Tupletlist[0]:3連音、Tupletlist[1]:5連音、Tupletlist[2]:7連音、Tupletlist[3]:9連音
Tupletlist =[Decimal(3)/2,Decimal(5)/4,Decimal(7)/8,Decimal(9)/8]
for i in range(0,4):
    AA = Measure[i]
    for Bitem in range(0,len(AA)):
        Rbeat = AA[Bitem].keys()[0]
        R = Rbeat / 128
        if R in BDic:
            addB[i].append([AA[Bitem][Rbeat],BDic[R],0])
        elif filter(lambda x: R*x in BDic,dotslist):
            DD = filter(lambda x: R*x in BDic,dotslist)
            dot = dotslist.index(DD[0])+1
            addB[i].append([AA[Bitem][Rbeat],BDic[DD[0]*R],dot])
        elif filter(lambda x :R*x in BDic,Tupletlist)
print addB 

[[[0, 'whole', 0]], [[0, 'whole', 0]], [[0, 'whole', 0]], [[1, 'eighth', 1], [1, '16th', 0], [0, 'eighth', 0], [1, 'eighth', 0], [0, '16th', 0], [1, '16th', 0], [1, '16th', 0], [1, '16th', 0], [1, 'quarter', 0]]]
